In [ ]:
#@title 📦 Könyvtárak telepítése
!pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers
print("✅ Könyvtárak telepítve")


In [ ]:
#@title 🚀 Qwen/Qwen3-Embedding-8B Script - Nagy Teljesítmény
import pandas as pd
import numpy as np
import gc
import json
import pyarrow as pa
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer
import torch
import tqdm
import time
import logging

# ============================================================
# KONFIGURÁLÁS - NAGY TELJESÍTMÉNY
# ============================================================

INPUT_CSV = "cleaned_data_for_embedding.csv"
OUTPUT_PARQUET = "processed_documents_with_embeddings.parquet"
MODEL_NAME = "Qwen/Qwen3-Embedding-8B"  # Pontos modellnév

# Optimalizált paraméterek tisztított szövegekhez
CHUNKSIZE = 5000
BATCH_SIZE = 8
MAX_TOKEN_LENGTH = 8192

# GPU optimalizáció
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("🏛️ EMBEDDING GENERÁTOR")
print("=" * 60)
print(f"Embedding Modell: {MODEL_NAME}")
print(f"Embedding Dimenzió: 8192")
print(f"Teljesítmény: Nagy")
print("=" * 60)
